In [ ]:
import csv
import json
import tensorflow as tf
import numpy as np
import urllib


# DO NOT CHANGE THIS CODE
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)


def solution_model():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv'
    urllib.request.urlretrieve(url, 'sunspots.csv')

    time_step = []
    sunspots = []

    # YOUR CODE HERE
    gpus = tf.config.experimental.list_physical_devices('GPU')
    
    physical_devices = tf.config.list_physical_devices('GPU')
    tf.config.experimental.set_memory_growth(gpus[0], enable=True)
    '''
    if gpus:
        try:
            tf.config.experimental.set_virtual_device_configuration(
                gpus[0],
                [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1200)]
            )
        except RuntimeError as e:
            print(e)
    '''

    
    early_stopping = tf.keras.callbacks.EarlyStopping(patience=10,
                                                      monitor='loss')
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss',
                                                     min_lr=1e-5,
                                                     patience=5,
                                                     mode='min')
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(monitor='mae',
                                                          filepath='saved_models/examQ5.h5',
                                                          save_best_only=True,
                                                          verbose=1)
    callbacks = [
        #myCallback(),
        #early_stopping,
        #reduce_lr,
        model_checkpoint
    ]
    
    with open('sunspots.csv') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        for row in reader:
            sunspots.append(float(row[2]))
            time_step.append(int(row[0]))
    # YOUR CODE HERE)

    series = np.array(sunspots)  # YOUR CODE HERE

    # DO NOT CHANGE THIS CODE
    # This is the normalization function
    min = np.min(series)
    max = np.max(series)
    series -= min
    series /= max
    time = np.array(time_step)

    # The data should be split into training and validation sets at time step 3000
    # DO NOT CHANGE THIS CODE
    split_time = 3000

    # YOUR CODE HERE
    time_train = time[:split_time]
    x_train = series[:split_time]
    time_valid = time[:split_time]
    x_valid = series[:split_time]
    # YOUR CODE HERE

    # DO NOT CHANGE THIS CODE
    window_size = 30
    batch_size = 32
    shuffle_buffer_size = 1000

    train_set = windowed_dataset(x_train, window_size=window_size, batch_size=batch_size,
                                 shuffle_buffer=shuffle_buffer_size)

    model = tf.keras.models.Sequential([
        # YOUR CODE HERE
        # Whatever your first layer is, the input shape will be [None,1] when using the Windowed_dataset above, depending on the layer type chosen
      
      tf.keras.layers.Conv1D(filters=32, kernel_size=5, strides=1, padding='causal', activation='relu', input_shape=[None, 1]),
      tf.keras.layers.LSTM(units=32, return_sequences=True),
      tf.keras.layers.LSTM(units=32, return_sequences=True),
      tf.keras.layers.Dense(units=30, activation='relu'),
      tf.keras.layers.Dense(units=30, activation='relu'),

      #tf.keras.layers.Conv1D(filters=64, kernel_size=3, strides=1, padding="causal", activation="relu", input_shape=[None, 1]),
      #tf.keras.layers.LSTM(64, return_sequences=True),
      #tf.keras.layers.LSTM(64),
      #tf.keras.layers.Dense(1),

      # YOUR CODE HERE
      tf.keras.layers.Dense(1)
    ])

    #model.compile(loss='mse', optimizer='SGD', metrics=['mae'])
    model.compile(loss=tf.keras.losses.Huber(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), metrics=['mae'])
    history = model.fit(train_set,
                        epochs=100,
                        verbose=1,
                        callbacks=callbacks)

    loss = history.history['loss']
    acc = history.history['mae']

    print("loss : ", loss[-1])
    print("mae : ", acc[-1])

    # loss :  0.007287612650543451
    # mae :  0.06163202226161957
    
    # Without lr reduction
    # loss :  0.0004591379838529974
    # mae :  0.01859680935740471
    
    # With lr reduction (Big model)
    #loss :  0.0019107466796413064
    # mae :  0.044232673943042755

    
    return model


if __name__ == '__main__':
    model = solution_model()
    
    

2022-10-15 16:59:46.651316: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-15 16:59:46.760615: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-15 16:59:46.761073: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-15 16:59:46.771832: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch 1/100


2022-10-15 16:59:54.589309: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8204


     93/Unknown - 9s 14ms/step - loss: 0.0040 - mae: 0.0631
Epoch 1: mae improved from inf to 0.06307, saving model to saved_models/examQ5.h5
93/93 [==============================] - 9s 19ms/step - loss: 0.0040 - mae: 0.0631
Epoch 2/100
90/93 [============================>.] - ETA: 0s - loss: 0.0021 - mae: 0.0467
Epoch 2: mae improved from 0.06307 to 0.04671, saving model to saved_models/examQ5.h5
93/93 [==============================] - 2s 16ms/step - loss: 0.0021 - mae: 0.0467
Epoch 3/100
92/93 [============================>.] - ETA: 0s - loss: 0.0021 - mae: 0.0468
Epoch 3: mae did not improve from 0.04671
93/93 [==============================] - 1s 14ms/step - loss: 0.0021 - mae: 0.0468
Epoch 4/100
93/93 [==============================] - ETA: 0s - loss: 0.0021 - mae: 0.0462
Epoch 4: mae improved from 0.04671 to 0.04622, saving model to saved_models/examQ5.h5
93/93 [==============================] - 2s 15ms/step - loss: 0.0021 - mae: 0.0462
Epoch 5/100
91/93 [=======================

93/93 [==============================] - 1s 14ms/step - loss: 0.0012 - mae: 0.0362
Epoch 35/100
92/93 [============================>.] - ETA: 0s - loss: 0.0012 - mae: 0.0356
Epoch 35: mae improved from 0.03622 to 0.03554, saving model to saved_models/examQ5.h5
93/93 [==============================] - 1s 14ms/step - loss: 0.0012 - mae: 0.0355
Epoch 36/100
93/93 [==============================] - ETA: 0s - loss: 0.0011 - mae: 0.0343
Epoch 36: mae improved from 0.03554 to 0.03427, saving model to saved_models/examQ5.h5
93/93 [==============================] - 1s 14ms/step - loss: 0.0011 - mae: 0.0343
Epoch 37/100
90/93 [============================>.] - ETA: 0s - loss: 0.0011 - mae: 0.0334
Epoch 37: mae improved from 0.03427 to 0.03341, saving model to saved_models/examQ5.h5
93/93 [==============================] - 1s 14ms/step - loss: 0.0011 - mae: 0.0334
Epoch 38/100
92/93 [============================>.] - ETA: 0s - loss: 0.0010 - mae: 0.0322
Epoch 38: mae improved from 0.03341 to 0.03

Epoch 66/100
92/93 [============================>.] - ETA: 0s - loss: 5.6932e-04 - mae: 0.0220
Epoch 66: mae did not improve from 0.02168
93/93 [==============================] - 2s 15ms/step - loss: 5.6891e-04 - mae: 0.0220
Epoch 67/100
91/93 [============================>.] - ETA: 0s - loss: 5.6182e-04 - mae: 0.0218
Epoch 67: mae did not improve from 0.02168
93/93 [==============================] - 2s 15ms/step - loss: 5.6139e-04 - mae: 0.0218
Epoch 68/100
91/93 [============================>.] - ETA: 0s - loss: 5.5904e-04 - mae: 0.0217
Epoch 68: mae did not improve from 0.02168
93/93 [==============================] - 2s 18ms/step - loss: 5.6094e-04 - mae: 0.0218
Epoch 69/100
74/93 [======================>.......] - ETA: 0s - loss: 5.3578e-04 - mae: 0.0209